# MLlib: Estatística Básica e Análise Exploratória de Dados

#### [Baseado em "Introduction to Spark with Python, by Jose A. Dianes"](https://github.com/jadianes/spark-py-notebooks)

Até agora usamos diferentes funções de mapeamento e agregação, em RDDs simples e chave / valor, a fim de obter estatísticas simples que nos ajudem a entender nossos conjuntos de dados. Neste caderno, introduziremos a biblioteca de aprendizado de máquina do Spark [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) através de sua funcionalidade básica de estatísticas, a fim de entender melhor nosso conjunto de dados. Usaremos os conjuntos de dados reduzidos de 10% [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) através do bloco de notas.

## Obtendo os dados e criando o RDD

Como fizemos em nosso primeiro notebook, usaremos o conjunto de dados reduzido (10 por cento) fornecido para a [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html), contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo Gzip que será baixado localmente.

In [ ]:
from urllib.request import urlretrieve

f = urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [ ]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## Local vectors

Um [local vector](https://spark.apache.org/docs/latest/mllib-data-types.html#local-vector) é geralmente usado como um tipo de base para RDDs no Spark MLlib. Um vetor local tem índices de tipos inteiros e baseados em 0 e valores de tipo duplo, armazenados em uma única máquina. O MLlib suporta dois tipos de vetores locais: denso e esparso. Um vetor denso é apoiado por um array duplo representando seus valores de entrada, enquanto um vetor esparso é apoiado por dois arrays paralelos: índices e valores.

Para vetores densos, o MLlib usa o Python *lists* ou o tipo *NumPy* `array`. O último é recomendado, então você pode simplesmente passar matrizes NumPy ao redor.

Para vetores esparsos, os usuários podem construir um objeto `SparseVector` a partir de vetores de coluna MLlib ou pass *SciPy*` scipy.sparse` se o SciPy estiver disponível em seu ambiente. A maneira mais fácil de criar vetores esparsos é usar os métodos de fábrica implementados em `Vetores`.

### An RDD of dense vectors

Vamos representar cada interação de rede em nosso conjunto de dados como um vetor denso. Para isso, vamos usar o tipo *NumPy* `array`.

In [ ]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

## Estatísticas resumidas

O MLlib do Spark fornece estatísticas de resumo de coluna para `RDD [Vector]` através da função [`colStats`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib. stat.Statistics.colStats) disponível em [`Statistics`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics). O método retorna uma instância de [`MultivariateStatisticalSummary`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.MultivariateStatisticalSummary), que contém as colunas wise *max*, *min*, *mean*, *variance* e *number of nonzeros*, bem como a *total count*. 

In [ ]:
from pyspark.mllib.stat import Statistics 
from math import sqrt 

# Compute column summary statistics.
summary = Statistics.colStats(vector_data)

print("Duration Statistics:")
print(" Mean: {}".format(round(summary.mean()[0],3)))
print(" St. deviation: {}".format(round(sqrt(summary.variance()[0]),3)))
print(" Max value: {}".format(round(summary.max()[0],3)))
print(" Min value: {}".format(round(summary.min()[0],3)))
print(" Total value count: {}".format(summary.count()))
print(" Number of non-zero values: {}".format(summary.numNonzeros()[0]))

### Estatísticas resumidas por label

A parte interessante das estatísticas de resumo, no nosso caso, vem de poder obtê-las pelo tipo de ataque de rede ou "label" em nosso conjunto de dados. Ao fazer isso, poderemos caracterizar melhor nossa variável dependente do conjunto de dados em termos do intervalo de valores das variáveis independentes.

Se quisermos fazer uma coisa dessas, poderíamos filtrar nossos labels RDD contendo chaves e vetores como valores. Para isso, precisamos apenas adaptar nossa função `parse_interaction` para retornar uma tupla com ambos os elementos.  

In [ ]:
def parse_interaction_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[41], np.array([float(x) for x in clean_line_split]))

label_vector_data = raw_data.map(parse_interaction_with_key)

O próximo passo não é muito sofisticado. Usamos `filter` no RDD para deixar de fora outros labels, mas o que queremos coletar estatísticas.

In [ ]:
normal_label_data = label_vector_data.filter(lambda x: x[0]=="normal.")

Agora podemos usar o novo RDD para chamar `colStats` nos valores.

In [ ]:
normal_summary = Statistics.colStats(normal_label_data.values())

E colete os resultados como fizemos antes.

In [ ]:
print("Duration Statistics for label: {}".format("normal"))
print(" Mean: {}".format(normal_summary.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(normal_summary.variance()[0]),3)))
print(" Max value: {}".format(round(normal_summary.max()[0],3)))
print(" Min value: {}".format(round(normal_summary.min()[0],3)))
print(" Total value count: {}".format(normal_summary.count()))
print(" Number of non-zero values: {}".format(normal_summary.numNonzeros()[0]))

Em vez de trabalhar com um par de chave/valor, poderíamos apenas filtrar nossos dados brutos divididos usando o label na coluna 41. Então, podemos analisar os resultados como fizemos antes. Isso funcionará também. No entanto, ter nossos dados organizados como pares chave/valor abrirá a porta para melhores manipulações. Já que `values()` é uma transformação em um RDD, e não uma ação, nós não executamos nenhum cálculo até chamarmos `colStats` de qualquer maneira.

Mas vamos incluir isso em uma função para que possamos reutilizá-la com qualquer label.

In [ ]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())

Vamos tentar com o "normal." label novamente.

In [ ]:
normal_sum = summary_by_label(raw_data, "normal.")

print("Duration Statistics for label: {}".format("normal"))
print(" Mean: {}".format(normal_sum.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(normal_sum.variance()[0]),3)))
print(" Max value: {}".format(round(normal_sum.max()[0],3)))
print(" Min value: {}".format(round(normal_sum.min()[0],3)))
print(" Total value count: {}".format(normal_sum.count()))
print(" Number of non-zero values: {}".format(normal_sum.numNonzeros()[0]))

Vamos tentar agora com algum ataque de rede. Nós temos todos eles listados [aqui](http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types). 

In [ ]:
guess_passwd_summary = summary_by_label(raw_data, "guess_passwd.")

print("Duration Statistics for label: {}".format("guess_password"))
print(" Mean: {}".format(guess_passwd_summary.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(guess_passwd_summary.variance()[0]),3)))
print(" Max value: {}".format(round(guess_passwd_summary.max()[0],3)))
print(" Min value: {}".format(round(guess_passwd_summary.min()[0],3)))
print(" Total value count: {}".format(guess_passwd_summary.count()))
print(" Number of non-zero values: {}".format(guess_passwd_summary.numNonzeros()[0]))

Podemos ver que esse tipo de ataque é mais curto do que uma interação normal. Poderíamos criar uma tabela com estatísticas de duração para cada tipo de interação em nosso conjunto de dados. Primeiro, precisamos obter uma lista de rótulos conforme descrito na primeira linha [aqui](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names).     

In [ ]:
label_list = ["back.","buffer_overflow.","ftp_write.","guess_passwd.",
              "imap.","ipsweep.","land.","loadmodule.","multihop.",
              "neptune.","nmap.","normal.","perl.","phf.","pod.","portsweep.",
              "rootkit.","satan.","smurf.","spy.","teardrop.","warezclient.",
              "warezmaster."]

Em seguida, obtemos uma lista de estatísticas para cada marcador.

In [ ]:
stats_by_label = [(label, summary_by_label(raw_data, label)) for label in label_list]

Agora, obtemos a coluna *duration*, primeiro em nosso conjunto de dados (ou seja, índice 0).

In [ ]:
duration_by_label = [ 
    (stat[0], np.array([float(stat[1].mean()[0]), float(sqrt(stat[1].variance()[0])), float(stat[1].min()[0]), float(stat[1].max()[0]), int(stat[1].count())])) 
    for stat in stats_by_label]

Que podemos colocar em um dataframe do Pandas.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

stats_by_label_df = pd.DataFrame.from_items(duration_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

E mostrar ele.

In [ ]:
print("Duration statistics, by label")
stats_by_label_df

Para reutilizar este código e obter um dataframe de qualquer variável em nosso conjunto de dados, definiremos uma função.

In [ ]:
def get_variable_stats_df(stats_by_label, column_i):
    column_stats_by_label = [
        (stat[0], np.array([float(stat[1].mean()[column_i]), float(sqrt(stat[1].variance()[column_i])), float(stat[1].min()[column_i]), float(stat[1].max()[column_i]), int(stat[1].count())])) 
        for stat in stats_by_label
    ]
    return pd.DataFrame.from_items(column_stats_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

Vamos tentar por *duration* novamente.

In [ ]:
get_variable_stats_df(stats_by_label,0)

Agora, para a próxima coluna numérica no conjunto de dados, *src_bytes*.

In [ ]:
print("src_bytes statistics, by label")
get_variable_stats_df(stats_by_label,1)

E assim por diante. Reutilizando as funções `summary_by_label` e` get_variable_stats_df` podemos realizar algumas análises de dados exploratórias em grandes conjuntos de dados com o Spark.

## Correlações

O MLlib do Spark suporta [Pearson's](http://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient) e [Spearman's](http://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) para calcular métodos de correlação entre pares entre muitas séries. Ambos são fornecidos pelo método `corr` no pacote` Statistics`.  

Nós temos duas opções como entrada. Dois `RDD [Double]` s ou um `RDD [Vector]`. No primeiro caso, a saída será um valor de 'Double', enquanto no segundo uma matriz de correlação inteira. Devido à natureza de nossos dados, obteremos o segundo.

In [ ]:
from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data, method="spearman")

Quando tivermos as correlações prontas, podemos começar a inspecionar seus valores.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

col_names = ["duration","src_bytes","dst_bytes","land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

corr_df

Nós usamos um *Pandas* `DataFrame` aqui para renderizar a matriz de correlação de uma forma mais abrangente. Agora queremos as variáveis altamente correlacionadas. Para isso, fazemos um pouco de manipulação de dataframe.

In [ ]:
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

### Conclusões e possíveis sugestões de seleção de modelos

O dataframe anterior nos mostrou quais variáveis são altamente correlacionadas. Mantivemos apenas essas variáveis com pelo menos uma correlação forte. Podemos usar como quisermos, mas um bom caminho poderia ser fazer uma seleção de modelos. Ou seja, se tivermos um grupo de variáveis altamente correlacionadas, podemos manter apenas uma delas para representar o grupo sob a suposição de que elas transmitem informações semelhantes como preditores. Reduzir o número de variáveis não melhorará a precisão do nosso modelo, mas tornará mais fácil de entender e também mais eficiente de calcular.

Por exemplo, a partir da descrição da  [KDD Cup 99 task](http://kdd.ics.uci.edu/databases/kddcup99/task.html) sabemos que a variável `dst_host_same_src_port_rate` referencia a porcentagem dos últimos 100 conexões para a mesma porta, para o mesmo host de destino. Em nossa matriz de correlação (e quadros de dados auxiliares) descobrimos que este é altamente e positivamente correlacionado com `src_bytes` e` srv_count`. O primeiro é o número de bytes enviados da origem ao destino. O último é o número de conexões para o mesmo serviço que a conexão atual nos últimos 2 segundos. Podemos decidir não incluir `dst_host_same_src_port_rate` em nosso modelo se incluirmos os outros dois, como uma forma de reduzir o número de variáveis e, posteriormente, melhor interpretar nossos modelos.

Mais tarde, nesses portáteis dedicados a construir modelos preditivos, usaremos essas informações para construir modelos mais interpretáveis.